# Importing packages

In [1]:
import os

import pandas as pd
import numpy as np

import cv2
from glob import glob as glob
from pathlib import Path

In [18]:
crack_prodir = Path("../data/processed/pro_crack_img")

In [37]:
def scale_thresholds(img_width, img_height, base_img_size=(227, 227)):
    base_area = base_img_size[0] * base_img_size[1]
    new_area = img_width * img_height
    area_scale = base_area / new_area
    width_scale = img_width / base_img_size[0]

    return {
        "area_thresh_low": 0.078 * area_scale,
        "area_thresh_high": 0.106 * area_scale,
        "width_thresh": int(40 * width_scale),
    }

def analyze_crack_intensity_v3(binary_img, thresholds):
    img_height, img_width = binary_img.shape
    img_area = img_height * img_width
    print(f"Image Size: {img_height} x {img_width}")
    contours, _ = cv2.findContours(
        binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    crack_features = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 10:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        bbox = binary_img[y : y + h, x : x + w]
        white_pixels = cv2.countNonZero(bbox)
        crack_ratio = white_pixels / img_area
        print(f"Crack Ratio: {crack_ratio:.4f}")
        local_width = min(w, h)
        print(f"Local Width: {local_width}")
        print(f"Area: {area}") 
        # Intensity classification logic
        if (crack_ratio < thresholds["area_thresh_low"] or local_width < thresholds["width_thresh"] ):
            intensity = 0  # Low
        elif (thresholds["area_thresh_low"]<= crack_ratio < thresholds["area_thresh_high"]) or (local_width < thresholds["width_thresh"] * 1.5):
            intensity = 1  # Medium
        else:
            intensity = 2  # High
        print(f"Intensity: {intensity}")
        crack_features.append({"bbox": (x, y, w, h), "intensity": intensity})

    return crack_features


In [44]:
def visualize_crack_intensity(img, crack_features):
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    for feat in crack_features:
        intensity = max((feat["intensity"] for feat in crack_features), default=0)
        
        # Set color based on intensity level
        if intensity == 0:
            color = (0, 255, 0)      # Green for Low
        elif intensity == 1:
            color = (0, 255, 255)    # Yellow for Medium
        else:
            color = (0, 0, 255)      # Red for High

        h, w = img.shape
        cv2.rectangle(output, (0, 0), (w - 1, h - 1), color, 4)
        cv2.putText(output, f"Intensity: {'Low' if intensity == 0 else 'Medium' if intensity == 1 else 'High'}", (10, 25), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 2)

    return output

In [45]:
# binary_img = your_preprocessed_crack_image
crack_images = list(crack_prodir.glob("*.jpg"))[80:90]

for img_path in crack_images:
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    
    img_height, img_width = img.shape
    thresholds = scale_thresholds(img_width, img_height)

    # Step 2: Analyze crack intensities
    crack_features = analyze_crack_intensity_v3(img, thresholds)

    # Step 3: Visualize the results
    vis = visualize_crack_intensity(img, crack_features)
    
    cv2.imshow(f"Crack Intensity - {img_path.name}", vis)
    cv2.waitKey(0)  # Wait for key press before moving to next image

cv2.destroyAllWindows()

Image Size: 227 x 227
Crack Ratio: 0.1070
Local Width: 128
Area: 4827.0
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.1184
Local Width: 160
Area: 8200.5
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.1120
Local Width: 107
Area: 5121.0
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.0084
Local Width: 32
Area: 342.5
Intensity: 0
Crack Ratio: 0.1496
Local Width: 151
Area: 8285.5
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.1115
Local Width: 160
Area: 6439.5
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.0002
Local Width: 5
Area: 10.5
Intensity: 0
Crack Ratio: 0.0536
Local Width: 99
Area: 2326.5
Intensity: 0
Crack Ratio: 0.1389
Local Width: 88
Area: 6859.0
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.1096
Local Width: 150
Area: 6295.5
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.1407
Local Width: 142
Area: 6436.0
Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.1075
Local Width: 64
Area: 4920.5
Intensity: 2
Crack Ratio: 0.1591
Local Width: 120
Area: 7743.5
Inten